# Aquisição Automática de Imagens com Grade (Grid)

## Instalação das Bibliotecas

In [ ]:
#!pip install earthengine-api --quiet
#!pip install rasterio --quiet
#!pip install geopandas --quiet
#!pip install requests --quiet
#!pip install folium --quiet

## Importações

In [ ]:
import ee
import os
import random
import datetime
import numpy as np
import rasterio
from rasterio import features
import geopandas as gpd
from shapely.geometry import box
import shutil
import requests
import json
import folium

## Montagem do Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Autenticação para o Google Earth Engine

In [ ]:
try:
  ee.Authenticate()
  ee.Initialize(project='761409644000')    # Substitua pelo seu ID de projeto do GEE
  print("Google Earth Engine inicializado com sucesso.")
except Exception as e:
  print(f"Erro ao inicializar o Google Earth Engine: {e}")
  print("Por favor, certifique-se de ter executado 'ee.Authenticate()' e seguido as instruções.")
  exit()

Google Earth Engine inicializado com sucesso.


## Parâmetros

In [ ]:
# --- Parâmetros de Download ---
output_folder_drive = "grid_rj" # Nome da pasta no Google Drive para salvar as imagens
area_size_meters = 1000 # Tamanho da célula da grade em metros (1000m = 1km)
start_date = "2023-01-01"
end_date = "2024-12-31"
max_cloud_coverage_s2 = 20 # Cobertura máxima de nuvens para Sentinel-2 (em %)
output_scale_meters = 10 # Resolução da imagem de saída em metros por pixel

# --- Parâmetros para a Grade (Grid) ---
# ATENÇÃO: Coloque abaixo o nome exato do seu arquivo .shp que você fez upload para o Colab
shapefile_path = '/content/drive/MyDrive/grid_rj/Mun_RJ.shp'
# ATENÇÃO: Nome EXATO do município como está na tabela de atributos do shapefile
municipality_to_process = 'RIO DE JANEIRO'

# Cria o diretório de saída, se não existir
os.makedirs(output_folder_drive, exist_ok=True)

# Nome do arquivo de saída para a grade
output_grid_filename = f"grid_{municipality_to_process.replace(' ', '_')}_{area_size_meters}m.shp"

## Funções Auxiliares

In [ ]:
def create_grid_for_municipality(aoi_shapefile_path, municipality_name, cell_size_meters):
    """
    Cria uma grade de polígonos sobre um MUNICÍPIO específico a partir de um shapefile.

    Args:
        aoi_shapefile_path (str): O caminho para o arquivo shapefile dos municípios.
        municipality_name (str): O nome do município para filtrar a AOI (ex: 'Rio de Janeiro').
        cell_size_meters (float): O tamanho do lado de cada célula da grade, em metros.

    Returns:
        geopandas.GeoDataFrame: Um GeoDataFrame contendo a grade de polígonos
                                que se sobrepõe ao município.
    """
    print(f"A criar grade para o município de '{municipality_name}'...")

    try:
        brasil_gdf = gpd.read_file(aoi_shapefile_path)
    except Exception as e:
        print(f"ERRO ao ler o shapefile: {e}")
        print(f"Verifique se o arquivo '{shapefile_path}' (e seus arquivos auxiliares .shx, .dbf etc.) foi carregado corretamente.")
        return None

    # Tenta filtrar pela coluna 'NM_MUN'. Se não existir, avisa o usuário.
    if 'NM_MUNICIP' not in brasil_gdf.columns:
        print("ERRO: A coluna 'NM_MUNICIP' não foi encontrada no shapefile. Verifique o nome da coluna que contém os nomes dos municípios.")
        return None

    municipality_gdf = brasil_gdf[brasil_gdf['NM_MUNICIP'] == municipality_name]

    if municipality_gdf.empty:
        print(f"ERRO: Município '{municipality_name}' não encontrado. Verifique se o nome está escrito exatamente como na tabela do shapefile.")
        return None

    municipality_geometry = municipality_gdf.unary_union
    municipality_utm = gpd.GeoSeries([municipality_geometry], crs="EPSG:4674").to_crs("EPSG:31983")
    min_x, min_y, max_x, max_y = municipality_utm.total_bounds

    x_coords = np.arange(min_x, max_x + cell_size_meters, cell_size_meters)
    y_coords = np.arange(min_y, max_y + cell_size_meters, cell_size_meters)

    grid_cells = [box(x, y, x + cell_size_meters, y + cell_size_meters) for x in x_coords for y in y_coords]
    grid_gdf_utm = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:31983")

    municipality_gdf_utm = gpd.GeoDataFrame(geometry=municipality_utm, crs="EPSG:31983")
    intersecting_grid = gpd.sjoin(grid_gdf_utm, municipality_gdf_utm, how="inner", predicate="intersects")

    intersecting_grid = intersecting_grid.drop_duplicates('geometry')
    grid_wgs84 = intersecting_grid.to_crs("EPSG:4674")

    print(f"Grade criada com {len(grid_wgs84)} células de {cell_size_meters}x{cell_size_meters} metros.")
    return grid_wgs84

## Funções de Download do Sentinel

In [ ]:
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def get_sentinel2_image(geometry, start_date, end_date, max_cloud_coverage):
    s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_coverage)) \
        .map(mask_s2_clouds)

    if s2_collection.size().getInfo() == 0:
        return None

    s2_image = s2_collection.median().select(['B4', 'B3', 'B2', 'B8'])
    return s2_image if s2_image.bandNames().size().getInfo() > 0 else None

def get_sentinel1_image(geometry, start_date, end_date):
    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filter(ee.Filter.eq('resolution_meters', 10))

    if s1_collection.size().getInfo() == 0:
        return None

    s1_image = s1_collection.median().select(['VV', 'VH'])
    return s1_image if s1_image.bandNames().size().getInfo() > 0 else None

## Funções da Máscara OSM

In [ ]:
def get_geojson_from_overpass(geometry):
    coords_info = geometry.bounds().getInfo()['coordinates'][0]
    min_lon = min(p[0] for p in coords_info)
    min_lat = min(p[1] for p in coords_info)
    max_lon = max(p[0] for p in coords_info)
    max_lat = max(p[1] for p in coords_info)

    overpass_bbox = f"{min_lat},{min_lon},{max_lat},{max_lon}"
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json][timeout:60];
    (
      way["highway"~"^(primary|secondary|tertiary|residential)$"]({overpass_bbox});
    );
    (._;>;);
    out body;
    """

    try:
        response = requests.post(overpass_url, data={'data': overpass_query})
        response.raise_for_status()
        data = response.json()

        geojson_features = []
        if 'elements' in data and len(data['elements']) > 0:
            node_coords = {str(elem['id']): (elem['lon'], elem['lat']) for elem in data['elements'] if elem['type'] == 'node'}
            for element in data['elements']:
                if element['type'] == 'way':
                    coords_list = [node_coords.get(str(node_id)) for node_id in element['nodes'] if str(node_id) in node_coords]
                    if len(coords_list) >= 2:
                        feature = {
                            "type": "Feature", "properties": element.get('tags', {}),
                            "geometry": {"type": "LineString", "coordinates": coords_list}
                        }
                        geojson_features.append(feature)

        return geojson_features if geojson_features else None
    except Exception as e:
        print(f"Erro na API Overpass: {e}")
        return None

def get_rasterized_road_network_gee(geometry, scale_meters, area_name, output_folder_drive, geojson_features):
    if not geojson_features:
        print(f"Não há dados GeoJSON válidos para rasterizar para a área {area_name}.")
        return None
    try:
        ee_features = [ee.Feature(f) for f in geojson_features]
        road_feature_collection = ee.FeatureCollection(ee_features).map(lambda f: f.set('value', 1))

        empty_image = ee.Image(0).rename('roads').float()
        rasterized_roads_gee = empty_image.paint(road_feature_collection, 'value')

        description = f"mask_{area_name}"
        task = ee.batch.Export.image.toDrive(
            image=rasterized_roads_gee, description=description, folder=output_folder_drive,
            scale=scale_meters, region=geometry.bounds().getInfo()['coordinates'],
            fileFormat='GeoTIFF', formatOptions={'cloudOptimized': True}
        )
        task.start()
        print(f"Tarefa de Máscara '{description}' iniciada. ID: {task.id}")
        return task.id
    except Exception as e:
        print(f"Erro ao rasterizar rede viária com GEE: {e}")
        return None

## Geração e Exportação das Imagens

In [ ]:
# --- INÍCIO DO PROCESSO PRINCIPAL ---

# 1. Criar a grade para o município de interesse
grid_gdf = create_grid_for_municipality(shapefile_path, municipality_to_process, area_size_meters)

if grid_gdf is not None:
    print(f"\nIniciando o processo de exportação para as {len(grid_gdf)} células da grade.")
    print(f"As imagens serão baixadas APENAS para as células que contiverem rodovias.")
    print(f"Os arquivos serão salvos na pasta '{output_folder_drive}' no seu Google Drive.")
    print("Monitore o progresso na aba 'Tasks' do Google Earth Engine Code Editor.")

    # Salvar .shp do grid gerado
    try:
        grid_para_salvar.to_file(output_grid_filename)
        print(f"✅ Arquivo da grade salvo com sucesso como '{output_grid_filename}'")
        print("   Atualize a lista de arquivos (à esquerda) para ver o resultado.")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo da grade: {e}")

    # 2. Loop principal que itera sobre cada célula da grade
    for index, cell in grid_gdf.iterrows():
        print(f"\n--- Analisando Célula {index + 1}/{len(grid_gdf)} ---")

        # Converte a geometria da célula para um ee.Geometry
        coords = list(cell.geometry.exterior.coords)
        area_geometry = ee.Geometry.Polygon(coords)

        # Verifica se existem rodovias na célula antes de continuar
        geojson_features = get_geojson_from_overpass(area_geometry)

        if not geojson_features:
            print(f"Nenhuma rodovia encontrada na Célula {index + 1}. Pulando para a próxima...")
            continue

        # Se encontrou rodovias, inicia as tarefas de download
        print(f"Rodovias encontradas! Iniciando tarefas de download para a célula {index + 1}.")

        current_area_name = f"grid_cell_{index}"

        # Tarefa da Máscara de Rodovias
        get_rasterized_road_network_gee(area_geometry, output_scale_meters, current_area_name, output_folder_drive, geojson_features)

        # Tarefa do Sentinel-2
        s2_image_to_export = get_sentinel2_image(area_geometry, start_date, end_date, max_cloud_coverage_s2)
        if s2_image_to_export:
            s2_image_to_export = s2_image_to_export.clip(area_geometry)
            s2_description = f"sentinel2_{current_area_name}"
            s2_task = ee.batch.Export.image.toDrive(
                image=s2_image_to_export, description=s2_description, folder=output_folder_drive,
                scale=output_scale_meters, region=area_geometry.getInfo()['coordinates']
            )
            s2_task.start()
            print(f"Tarefa Sentinel-2 '{s2_description}' iniciada. ID: {s2_task.id}")
        else:
            print(f"Nenhuma imagem Sentinel-2 encontrada para a célula {index + 1}.")

        # Tarefa do Sentinel-1
        s1_image_to_export = get_sentinel1_image(area_geometry, start_date, end_date)
        if s1_image_to_export:
            s1_image_to_export = s1_image_to_export.clip(area_geometry)
            s1_description = f"sentinel1_{current_area_name}"
            s1_task = ee.batch.Export.image.toDrive(
                image=s1_image_to_export, description=s1_description, folder=output_folder_drive,
                scale=output_scale_meters, region=area_geometry.getInfo()['coordinates']
            )
            s1_task.start()
            print(f"Tarefa Sentinel-1 '{s1_description}' iniciada. ID: {s1_task.id}")
        else:
            print(f"Nenhuma imagem Sentinel-1 encontrada para a célula {index + 1}.")

    print("\nProcesso de inicialização de tarefas concluído para todas as células relevantes da grade!")
    print("Verifique o seu Google Drive e a aba 'Tasks' do Earth Engine para os resultados.")

else:
    print("\nA execução foi interrompida porque a grade não pôde ser criada.")

A criar grade para o município de 'RIO DE JANEIRO'...


/tmp/ipython-input-1411351449.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  municipality_geometry = municipality_gdf.unary_union


A saída de streaming foi truncada nas últimas 5000 linhas.
Rodovias encontradas! Iniciando tarefas de download para a célula 1016.
Tarefa de Máscara 'mask_grid_cell_1015' iniciada. ID: 5BGXZBMZDGI5VS22SFVELDS4
Tarefa Sentinel-2 'sentinel2_grid_cell_1015' iniciada. ID: OB32HCW2466GGTZDGAFEHC7N
Tarefa Sentinel-1 'sentinel1_grid_cell_1015' iniciada. ID: H7HOQJ5X4FUJXSZDRCIHCGI6

--- Analisando Célula 1017/1404 ---
Nenhuma rodovia encontrada na Célula 1017. Pulando para a próxima...

--- Analisando Célula 1030/1404 ---
Rodovias encontradas! Iniciando tarefas de download para a célula 1030.
Tarefa de Máscara 'mask_grid_cell_1029' iniciada. ID: YL6UUPXPZQOSWBEMGALJP3RF
Tarefa Sentinel-2 'sentinel2_grid_cell_1029' iniciada. ID: RJUSWRJK4KJ7Y63YAFNV2HXQ
Tarefa Sentinel-1 'sentinel1_grid_cell_1029' iniciada. ID: XW6PNJO7BJ3LYBLCONZODHLZ

--- Analisando Célula 1031/1404 ---
Nenhuma rodovia encontrada na Célula 1031. Pulando para a próxima...

--- Analisando Célula 1032/1404 ---
Nenhuma rodovia e